### <u> Importing Modules </u>

In [26]:
import pandas as pd
import numpy as np
import math

### <u>Retrieving TMS ID of Existing Unitholders Who Actively Traded In the Market.</u>
#### *Active Traders and non active traders are classified based on the report provided by cdsc*

In [27]:
tms_codes = pd.read_excel('Shareholder List As of 27.07.2023.xlsx')
tms_ids = tms_codes['tms_clean'].dropna().drop_duplicates()

### <u> Reading Floorsheet Data </u>

#### *Floorsheet Data Provided by NEPSE for the respective mutual fund.*

In [35]:
data= pd.read_excel('Floorsheet AFTER ADDING IPO DATA.xlsm')
data['CONTRACT_DATE'] = pd.to_datetime(data['CONTRACT_DATE'])
print(f'The total number of traders as per TMS {len(data.TMS_CLIENT_CODE.drop_duplicates())}')

The total number of traders as per TMS 8640


In [ ]:
all_info = []
for index, each_client in  enumerate(tms_ids):
    print(index, each_client)
    indi_data = data.copy()
    indi_data = indi_data[indi_data['TMS_CLIENT_CODE'] == each_client]
    indi_data.sort_values(by = ['CONTRACT_DATE'], ascending= True, inplace = True)
    indi_data['Quantity_buy_sell'] = np.where((indi_data['BUY_OR_SELL'] == "Buy") | (indi_data['BUY_OR_SELL'] == "IPO"), indi_data['CONTRACT_QUANTITY'], -indi_data['CONTRACT_QUANTITY'])
    condition = 0
    if len(indi_data) ==0:
        client_name = 'error'
        wacc = 'error'
        remarks = 'error'
    else:
        if set(indi_data['BUY_OR_SELL'].unique()).issubset(['Buy', 'IPO']) == True: #If client has shares from IPO and buy only, no sell
            dp_fee = len(indi_data['CONTRACT_DATE'].drop_duplicates())*25
            wacc = ((indi_data['Effective amount'].sum()+dp_fee)/indi_data['CONTRACT_QUANTITY'].sum())
            client_name = indi_data['ClientFullName'].iloc[0]
            remarks = 'IPO and buy only since inception'
        elif set(indi_data['BUY_OR_SELL'].unique()).issubset(['Sell', 'IPO']) == True:#If client has sell and IPO only, no buy
            wacc = 10
            client_name = indi_data['ClientFullName'].iloc[0]
            remarks = 'IPO and sell only since inception'
        else: # If client has IPO, buy, and sell trades
            indi_data =  indi_data[[ 'ClientFullName', 'TMS_CLIENT_CODE', 'BUY_OR_SELL', 'CONTRACT_DATE','Effective amount', 'Effective rate','CONTRACT_QUANTITY', 'Quantity_buy_sell' ]]
            indi_data['Cumsum_qty'] = indi_data['Quantity_buy_sell'].cumsum()
            indi_data['cumsum_amt'] = indi_data['Effective amount'].cumsum()

            if not(indi_data['Cumsum_qty'] == 0).any() == True: #Read below docstring
                """If there is no any zero values in cumsum_qty column it means
                    the entire positions was never sold off. Which means total sum of amt divided by total qty gives wacc."""
                if list(indi_data['BUY_OR_SELL'])[0] == 'IPO': #If the client has shares from IPO
                    dp = indi_data
                    dp = dp[dp['BUY_OR_SELL'] == 'Buy']
                    no_of_days_of_buy = len(dp['CONTRACT_DATE'].drop_duplicates())
                    total_dp_fee = no_of_days_of_buy * 25
                    buy_only_trades = indi_data[indi_data['BUY_OR_SELL'].isin(['IPO', 'Buy'])]
                    wacc = (buy_only_trades['Effective amount'].sum()+total_dp_fee)/buy_only_trades['CONTRACT_QUANTITY'].sum()
                    client_name =  indi_data['ClientFullName'].iloc[0]
                    remarks = 'Has IPO,buy, and sell, entire portfolio never liquidated'
                else: #If the client does not have ipo shares
                    dp = indi_data
                    dp = dp[dp['BUY_OR_SELL'] == 'Buy']
                    no_of_days_of_buy = len(dp['CONTRACT_DATE'].drop_duplicates())
                    total_dp_fee = no_of_days_of_buy * 25
                    buy_only_trades = indi_data[indi_data['BUY_OR_SELL'] == 'Buy']
                    wacc = ((buy_only_trades['Effective amount'].sum()+total_dp_fee)/buy_only_trades['Quantity_buy_sell'].sum())            
                    client_name = indi_data['ClientFullName'].iloc[0]
                    remarks = 'Has buy and sell shares no IPO, entire portfolio never sold'
            else: #If the cumsum_qty comprises 0 value which means position was entirely liquidated in the review period.
                for_dp_fee = indi_data.copy()
                if for_dp_fee.BUY_OR_SELL.iloc[0] == 'IPO':
                    for_dp_fee.iloc[0, -1] -= 25
                else:
                    pass
                for_dp_fee.replace('IPO', 'Buy', inplace = True)
                for_dp_fee['Cumsum_qty'] = for_dp_fee['Cumsum_qty'].replace(0, np.nan, regex=True)
                for_dp_fee['blocks'] = for_dp_fee['Cumsum_qty'].isnull().cumsum()
                for_dp_fee['keep_last'] = for_dp_fee['blocks'].duplicated(keep='last')
                for_dp_fee['qty_sum'] = for_dp_fee.groupby(['blocks', 'BUY_OR_SELL'])['CONTRACT_QUANTITY'].cumsum()
                for_dp_fee['amt_sum'] = for_dp_fee.groupby(['blocks', 'BUY_OR_SELL'])['Effective amount'].cumsum()

                remove_dups = for_dp_fee.copy()
                remove_dups.drop_duplicates(subset= 'CONTRACT_DATE', inplace = True)
                remove_dups['Buy_Count'] = remove_dups.groupby('blocks')['BUY_OR_SELL'].transform(lambda x: x.eq('Buy').sum())
                remove_dups.drop_duplicates(subset= 'blocks', inplace = True)
                remove_dups = remove_dups[['blocks', 'Buy_Count']]

                count_dp_fee = for_dp_fee.merge(remove_dups, on = 'blocks', how = 'left')
                count_dp_fee['DP Fee'] = count_dp_fee['Buy_Count'] * 25 #Don't apply dp fee for ipo----------->look here
                count_dp_fee['Total_buy_amount'] = count_dp_fee['amt_sum'] + count_dp_fee['DP Fee']
                count_dp_fee['WACC'] = count_dp_fee['Total_buy_amount']/count_dp_fee['qty_sum']
                count_dp_fee['block&signal'] = count_dp_fee['blocks'].astype(str) + count_dp_fee['BUY_OR_SELL'] 
                count_dp_fee = count_dp_fee[count_dp_fee['BUY_OR_SELL'] == 'Buy']
                count_dp_fee = count_dp_fee.drop_duplicates(subset= 'block&signal', keep = 'last')
                wacc = count_dp_fee['WACC'].iloc[-1]
                client_name = count_dp_fee['ClientFullName'].iloc[0]
                remarks = 'Has buy, sell, and IPO, and entire portfolio was liquidated in between and purchased again'
                condition = 3
                
        info = {'ClientID' : each_client,
                'Client Name' : client_name,
                'WACC'     : wacc,
                'Remarks' : remarks
               }
        all_info.append(info)
        
    writer = pd.ExcelWriter(f'WACC ALL (LATEST)/{each_client}.xlsx', engine='openpyxl')
    
    if condition == 0:
        indi_data.to_excel(writer, sheet_name='TRADES', index=False)
        pd.DataFrame([info]).to_excel(writer, sheet_name='WACC', index=False)
    else:
        count_dp_fee.to_excel(writer, sheet_name='TRADES', index=False)
        pd.DataFrame([info]).to_excel(writer, sheet_name='WACC', index=False)
    writer.save()
    writer.close()

In [ ]:
pd.DataFrame.from_dict(all_info).to_csv('WACC as on 30.07.2023.csv')